In [ ]:
# notebooks/03_data_merging.ipynb

import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import MinMaxScaler
import sys
import os

# Add src to path
sys.path.append(os.path.abspath('..'))
from src.utils import add_technical_indicators

# 1. SETUP & LOAD
print("Loading data...")
df_prices = pd.read_csv('../data/raw/hsi_price_history.csv')
df_sentiment = pd.read_csv('../data/processed/daily_sentiment.csv')

# --- FIX START: CLEAN UP YFINANCE DATA STRUCTURE ---
# yfinance often saves a 'Ticker' column or creates a MultiIndex (Price, Ticker)
# We flatten it and remove non-numeric columns immediately.

# If 'Ticker' exists as a column, drop it
if 'Ticker' in df_prices.columns:
    print("Dropping 'Ticker' column...")
    df_prices = df_prices.drop(columns=['Ticker'])

# If the dataframe has a MultiIndex columns (Level 0: Price, Level 1: Ticker), flatten it
if isinstance(df_prices.columns, pd.MultiIndex):
    print("Flattening MultiIndex columns...")
    df_prices.columns = df_prices.columns.get_level_values(0)

# Sometimes the first few rows are header metadata garbage, ensure 'Date' is real
# (This step depends on how exactly read_csv loaded it, but usually standard read_csv is fine)

# --- FIX END ---

# Convert dates
df_prices['Date'] = pd.to_datetime(df_prices['Date']).dt.tz_localize(None)
df_sentiment['Date'] = pd.to_datetime(df_sentiment['Date'])

# 2. MERGE
df_merged = pd.merge(df_prices, df_sentiment, on='Date', how='left')
df_merged['sentiment_score'] = df_merged['sentiment_score'].fillna(0)

# 3. ADD INDICATORS
print("Adding technical indicators...")
# Double check types before passing
# Force numeric on key columns
numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
for col in numeric_cols:
    if col in df_merged.columns:
        df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')

df_processed = add_technical_indicators(df_merged)

# Drop NaNs
df_processed.dropna(inplace=True)

# 4. SCALING
# Calculate the percentage change (Return)
df_processed['Return'] = df_processed['Close'].pct_change()

# The Target is NEXT day's return
df_processed['Target'] = df_processed['Return'].shift(-1)

# Drop NaNs created by shift/pct_change
df_processed.dropna(inplace=True)

# Save RAW data (Do not scale here!)
df_processed.to_csv('../data/processed/training_data.csv', index=False)
print(f"Success! Saved {len(df_processed)} rows with Returns target.")


Using GPU: NVIDIA GeForce RTX 3070 Ti Laptop GPU
Loading data...
Adding technical indicators...
Features to scale: ['Close', 'High', 'Low', 'Open', 'Volume', 'sentiment_score', 'MA_5', 'MA_10', 'MA_20', 'MA_50', 'RSI', 'Price_Change', 'Volume_Change', 'Volatility']
Success! Saved 2708 rows of RAW data.


c:\Users\Raynell\Desktop\fyp_code\src\utils.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.fillna(0, inplace=True)
